# OpenAI Key Setting

In [ ]:
import getpass

api_key = getpass.getpass("Please enter your input API KEY :")
organization_id = getpass.getpass("Please enter your input ORGANIZATION ID :")


# Import Pakage and Data Setting

### LangChain 사용환경 설정

* Langchain과 langchain-openai 라이브러리 패키지 설치  
  -  pip install langchain
  -  pip install langchain-openai  
* langchain python라이브러리로 프롬프트, 에이전트, 체인 관련 패키지 모음  
  - pip install langchainhub

In [ ]:
!pip install langchain
!pip install langchain-openai
!pip install langchainhub
!pip install langchain_community
!pip install langchain-core
!pip install wikipedia

In [ ]:
from langchain import hub  # 다양한 prompt를 가져오거나 등록해서 사용할 수 있는 패키지

from langchain_openai import ChatOpenAI  # for chat model
from langchain_openai import OpenAI     #for LLM
from langchain_core.pydantic_v1 import BaseModel, Field  # 출력 포맷을 지정을 위해 사용하는 클래스와 메서드 집합
from langchain.schema import (
    AIMessage,
    HumanMessage,
    SystemMessage
)
from langchain_core.prompts import (
    ChatPromptTemplate,
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    PipelinePromptTemplate,
    FewShotPromptTemplate
)
from langchain_core.output_parsers import (
    StrOutputParser,
    CommaSeparatedListOutputParser,
    JsonOutputParser
)

# 1. 기본 LLM

*   ChatOpenAI : 모델을 불러오는 클래스
*   ChatPromptTemplate : prompt 템플릿을 제공해주는 클래스
*   ChatPromptTemplate.from_template() : 문자열 형태의 템플릿을 인자로 받아, 해당 형식에 맞는 프롬프트 객체를 생성
*   StrOutputParaser : 모델을 출력을 문자열 형태로 파싱하여 최종 결과를 반환
*   invoke : chain을 실행하는 메서드
*   batch : Bach Prompt를 구성하고 LLM 호출을 방법

In [ ]:
# OpenAI()를 이용하여 클라이언트 인스턴스를 생성할 때 OPENAI_API_KEY, organization_id, project_id를 입력하여 인스턴스를 생성
llm = OpenAI(
    api_key=api_key,
    organization=organization_id,
)

In [ ]:
prompt = f"""
  You are an expert in Generative AI . Answer the question.
  <Question> : LangChain을 사용해서 언어모델로 서비스를 만들어보려고 해. LangChain API을 처음 사용하는 사람은 무엇부터 해야하지?
"""

In [ ]:
print(llm.invoke(prompt))

### 1-1. Batch Prompt
- Btach Prompt를 구성하여 LLM 호출  

- 일괄 처리: 여러 개의 입력(prompt)을 한 번에 모델에게 보내서 여러 개의 출력(response)을 받는 것
- 효율성 향상: 한 번에 여러 개의 작업을 처리함으로써 모델의 효율성을 높이고 처리 시간을 단축
- 일관성 유지: 동일한 환경에서 여러 개의 작업을 처리하여 결과의 일관성을 유지

In [ ]:
prompts = [
    "한국의 수도가 어디야?",
    "미국의 수도가 어디야?",
    "일본의 수도가 어디야?"
]

In [ ]:
print(llm.batch(prompts))

### 1-2. Token 사용량 추적 방법
   - Langchain 에서는 LLM 제공자가 제공하는 콜백함수를 통해서 LLM 호출건별 토큰 수를 카운트 할 수 있다.

   -  openai에서 제공하는 callback 함수를 이용하여 토큰수를 출력하는 코드 :  
      "with get_openai_callback() as callback:” 코드 블럭내에 LLM 호출 코드를 작성하면 LLM 을 호출한후에, callback에 호출에 대한 메타 정보를 담아서 리턴한다.

In [ ]:
from langchain.callbacks import get_openai_callback

with get_openai_callback() as callback:
    prompt = "LangChain을 사용해서 언어모델로 서비스를 만들어보려고 해. LangChain API을 처음 사용하는 사람은 무엇부터 해야하지?"
    llm.invoke(prompt)

In [ ]:
print(callback)

In [ ]:
print("Total Tokens:",callback.total_tokens)

### 1-3. ChatOpenAI
* OpenAI module  
  주로 텍스트 생성, 문서 요약, 질문 응답 등의 일반적인 언어 모델 기능을 제공  
  API 호출 시 prompt를 주고 이에 대한 모델의 응답을 받는 방식으로 작동, 특정 텍스트에 대해 이어지는 내용을 생성
  
  
* ChatOpenAI module  
  챗 인터페이스를 활용, 대화의 컨텍스트를 포함한 메시지를 주고 이에 대한 모델의 응답을 받는 방식으로 작동  
  이전 대화를 기억하고 이에 따라 응답하는 챗봇을 만들 때 사용

In [ ]:
chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    api_key=api_key,
    organization=organization_id                        
)

In [ ]:
# 직접 Prompt를 전달하는 방식
print(chat_model.invoke("hello there?"))

In [ ]:
# ChatModel에서 HumanMessage에 Prompt를 할당하여 호출하는 방식
prompt = "What would be a good company name for a company that makes AI solutions for Semiconductor manufacturing industry?"
messages = [HumanMessage(content=prompt)]

print("OpenAI using messags templetes ->\n", llm.invoke(messages))
print("ChatOpenAI using messags templetes ->\n", chat_model.invoke(prompt))
print("ChatOpenAI using messags templetes ->\n", chat_model.invoke(messages))

In [ ]:
# Prompt Templete와 LangChain 이용하여 호출하는 방식
prompt = ChatPromptTemplate.from_template("You are an expert in Generative AI . Answer the question. <Question>: {question}")
print(type(prompt))
print(prompt)

In [ ]:
chain = prompt | llm
print(chain.invoke({"question":"도체 제조 산업에서 거대언어 모델을 활용한 대표적인 사례를 알려줘"}))

In [ ]:
chain = prompt | chat_model
print(chain.invoke({"question":"도체 제조 산업에서 거대언어 모델을 활용한 대표적인 사례를 알려줘"}))

# 2. Prompt Template

*  PromptTemplate은 문자열 프롬프트에 대한 템플릿을 생성하는 데 사용
*  기본적으로 템플릿 작성에는 Python의 str.format 구문을 사용

In [ ]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    api_key=api_key,
    organization=organization_id                        
)

In [ ]:
# 기본적인 Prompt Template 형태
prompt = PromptTemplate.from_template("What is a good name for a company that make {product}")
print(type(prompt), prompt)

In [ ]:
text = prompt.format(product="Semiconductor")
print(type(text), text)

In [ ]:
chain = prompt | llm
print(chain.invoke("Semiconductor"))

In [ ]:
chain = prompt | llm | StrOutputParser() # OutputParser를 이용해 결과만 출력
print(chain.invoke("Semiconductor"))

In [ ]:
# 다중 입력 형태의 Template
prompt = PromptTemplate.from_template("안녕하세요, 제 이름은 {name}이고, 나이는 {age}살입니다.")
print(type(prompt), prompt)

In [ ]:
text = prompt.format(name="James", age="30")
print(type(text), text)

In [ ]:
# 여러 Template와 문자열의 결함
combined_prompt = (
    prompt
    + PromptTemplate.from_template(" 그리고 저는 {city}에 살아요")
    + "\n\n{language}로 번역해주세요"
)
print(type(combined_prompt), combined_prompt)

In [ ]:
text = combined_prompt.format(name="James", age="30", city="NewYork", language="English")
print(type(text), text)

In [ ]:
chain = combined_prompt | llm | StrOutputParser()
print(chain.invoke({"name":"James", "age":30, "city":"NewYork", "language":"English"}))

# 3. Chat Prompt Template
*  각 채팅 메시지는 content와 추가적인 매개변수 role이 연결
*  OpenAI Chat Completions API 에서 채팅 메시지는 AI Assitant, Human,  System 역할과 연결

* ChatPromptTemplate는 대화형 상황에서 여러 메시지 입력을 기반으로 단일 메시지 응답을 생성하는 데 사용.

  - ChatPromptTemplate.from_messages : 메시지 리스트(혹은 튜플)을 기반으로 프롬프트를 구성함.
  - ChatPromptTemplate.format_messages : 사용자의 입력을 프롬프트에 동적으로 삽입하여, 최종적으로 대화형 상황을 반영한 메시지 리스트를 생성

In [ ]:
chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    api_key=api_key,
    organization=organization_id                        
)

In [ ]:
chat_template = ChatPromptTemplate.from_messages(
    [   # (Role, Content) -> Tuple 형태
        ("system", "You are a helpful AI bot. Your name is {name}."),  # <class 'langchain_core.messages.system.SystemMessage'> 타입 지정
        ("human", "Hello, how are you doing?"),                        # <class 'langchain_core.messages.human.HumanMessage'> 타입 지정
        ("ai", "I'm doing well, thanks!"),                             # <class 'langchain_core.messages.ai.AIMessage'> 타입 지정
        ("human", "{user_input}"),
        # Role Type : 'human', 'user', 'ai', 'assistant', 'system'.
    ]
)

messages = chat_template.format_messages(name="Bob", user_input="What is your name?")
print(type(messages[0]), type(messages[1]), type(messages[2]))
print(messages)

In [ ]:
chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that translates {input_language} to {output_language}."),
    ("human", "{text}")
])

messages = chat_prompt.format_messages(
    input_language="English",
    output_language="Korean",
    text="I am an AI expert. I love chatting with ChatGPT."
)

print(messages)
print(chat_model.invoke(messages))

In [ ]:
# Chat Model이 아닐 시 ChatPromptTemplate.from_messages 사용 불가
from openai import NotFoundError

llm = OpenAI(
    model="gpt-3.5-turbo-0125",
    api_key=api_key,
    organization=organization_id                        
)

try:
    llm.invoke(messages)
except NotFoundError as e:
    print("[Err Log]", e)


### 3-1. Template Lang Chain 응용

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "너는 월드 클래스 급의 문서 작성 전문가야. 영어로 작성해줘"),
    ("user", "{user_input}")
])
output_parser = StrOutputParser()

chain = prompt | chat_model | output_parser
print(chain.invoke({"user_input": "이용하여 반도체 제조공정에 대한 교육 자료를 중학생이 이해할 수 있는 수준으로 작성해줘"}))

In [ ]:
chain_prompt = ChatPromptTemplate.from_messages([
    ("system", "넌 입력의 내용을 전체로 2문장으로 한글로 요약해주는 시스템이야"),
    ("user", "{context}")
])

trans_chain = {"context":chain} | chain_prompt | chat_model | StrOutputParser()
print(trans_chain.invoke(input="반도체 제조공정에 대한 교육 자료를 중학생이 이해할 수 있는 수준으로 작성해줘"))

# 4. LangChain Memory

* 대화 기록을 저장하고, 문맥으로 활용
* langchain.memory 패키지에서 ChatMessageHistory import 하여 사용

  - ChatMessageHistory() : 대화기록을 유지하기 위해 랭체인이 제공하는 클래스
    - add_user_messgge() : 사용자 메시지를 대화기록 객체에 추가하기 위한 메서드
    - add_user_messgge() : ai 가 답변한 대화내용을 대화기록 개첵에 추가하는 메서드

In [ ]:
chat_model = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    api_key=api_key,
    organization=organization_id                        
)

In [ ]:
from langchain.memory import ChatMessageHistory

history = ChatMessageHistory()
history.add_user_message("Where is the top 3 popular space for tourist in Seoul?")

aiMessage = chat_model.invoke(history.messages)
print(aiMessage.content)
history.add_ai_message(aiMessage.content)

In [ ]:
# 현재까지 대화내역 확인
print(f"<history Count : {len(history.messages)}>", history.messages)

In [ ]:
history.add_user_message("Which transport can I use to visit the places?")

aiMessage = chat_model.invoke(history.messages)
history.add_ai_message(aiMessage.content)

In [ ]:
# 추가 대화내역 확인
print(f"<history Count : {len(history.messages)}>", history.messages)

# 5. LLM Chain과 PromptTemplate 사용

 - LLM Chain은 프롬프트 템플릿을 LLM에 합쳐서 컴포넌트화 한 것
 - 입력값으로 문자열을 넣으면 프롬프트 템플릿에 의해서 프롬프트가 자동으로 완성
 - LLM 모델을 호출하여 텍스트 출력을 내주는 기능

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import LLMChain

chat_model = OpenAI(
    api_key=api_key,
    organization=organization_id
)

prompt = PromptTemplate.from_template("{city}에서 가장 유명한 관광지 3개를 추천해줘")

In [ ]:
chain = LLMChain(llm=chat_model, prompt=prompt)

In [ ]:
print(chain.run("서울"))

# 6. Sequential Chain 이용

* LLMChain 컴포넌트를 만들고 난 뒤, LLMChain들을 서로 연결하기 위해 여러 Chain을 순차적으로 연결하게 해주는 컴포넌트인 SequentialChain을 사용

    - 아래 예제는 먼저 도시 이름 {city}을 입력 받은 후에, 첫번째 chain에서 그 도시의 유명한 관광지 이름을 {place}로 출력하도록 한다.
    - 다음 두번째 chain에서는 관광지 이름 {place}를 앞의 chain에서 입력 받고, 추가적으로 교통편 {transport}를 입력받아서, 그 관광지까지 가기 위한 교통편 정보를 최종 출력으로 제공한다.

    - 이때  첫번쨰 chain1 생성시에 output_key를 명시적으로 place로 지정해 준 것에 주의하자.


In [ ]:
from langchain.chains import SequentialChain

chat_model = OpenAI(
    api_key=api_key,
    organization=organization_id                        
)

In [ ]:
prompt1 = PromptTemplate.from_template("{city}에서 가장 유명한 관광명소를 추가적인 설명없이 장소 이름만으로 추천해줘.")
chain1 = LLMChain(llm=chat_model, prompt=prompt1, output_key="place", verbose=True) # verbose : Debug Option

In [ ]:
print("[result]\n", chain1.invoke("서울"))

In [ ]:
prompt2 = PromptTemplate.from_template("{place}을 {transport} 편으로 가는 방법을 알려줄래?")
chain2 = LLMChain(llm=chat_model, prompt=prompt2, verbose=True)

In [ ]:
print("[result]\n", chain2.run({"place":"서울", "transport":"지하철"}))

In [ ]:
chain = SequentialChain(chains=[chain1,chain2], input_variables=["city","transport"], verbose=True)
print("[result]\n", chain.run({'city':'인천','transport':'전철'}))

# 7. Prompt Serialization

 프롬프트를 수시로 변경해야 하는 경우에 프롬프트 템플릿을 별도의 파일로 저장하여, 애플리케이션에서 로드하여 사용하는 방법

* 프롬프트 템플릿을 저장하는 방법
 - PrompteTemplate객체를 생성한후에 save(“{JSON 파일명}”)을 해주면 해당 템플릿이 파일로 저장

* 저장된 프롬프트 템플릿을 Load하여 사용하기
  - 저장된 템플릿은 langchain.prompts 패키지의 load_prompt 함수를 이용하여 다시 불러와 사용 가능

In [ ]:
template = PromptTemplate.from_template(
    "Tell me a {adjective} {topic} in {city} in 300 characters."
)
print(template)

In [ ]:
template.save("./template.json")

In [ ]:
from langchain.prompts import load_prompt

loaded_template = load_prompt("template.json")
print(loaded_template)

In [ ]:
prompt = loaded_template.format(adjective="popular", topic="cafe", city="San francisco")
print(prompt)

# 8. Few Shot Prompt Template

In [ ]:
examples = [
    {
        "question": "스티브 잡스와 아인슈타인 중 누가 더 오래 살았나요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 스티브 잡스는 몇 살에 사망했나요?
                  중간 답변: 스티브 잡스는 56세에 사망했습니다.
                  추가 질문: 아인슈타인은 몇 살에 사망했나요?
                  중간 답변: 아인슈타인은 76세에 사망했습니다.
                  최종 답변은: 아인슈타인
                  """,
    },
    {
        "question": "네이버의 창립자는 언제 태어났나요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 네이버의 창립자는 누구인가요?
                  중간 답변: 네이버는 이해진에 의해 창립되었습니다.
                  추가 질문: 이해진은 언제 태어났나요?
                  중간 답변: 이해진은 1967년 6월 22일에 태어났습니다.
                  최종 답변은: 1967년 6월 22일
                  """,
    },
    {
        "question": "율곡 이이의 어머니가 태어난 해의 통치하던 왕은 누구인가요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 율곡 이이의 어머니는 누구인가요?
                  중간 답변: 율곡 이이의 어머니는 신사임당입니다.
                  추가 질문: 신사임당은 언제 태어났나요?
                  중간 답변: 신사임당은 1504년에 태어났습니다.
                  추가 질문: 1504년에 조선을 통치한 왕은 누구인가요?
                  중간 답변: 1504년에 조선을 통치한 왕은 연산군입니다.
                  최종 답변은: 연산군
                  """,
    },
    {
        "question": "올드보이와 기생충의 감독이 같은 나라 출신인가요?",
        "answer": """
                  이 질문에 추가 질문이 필요한가요: 예.
                  추가 질문: 올드보이의 감독은 누구인가요?
                  중간 답변: 올드보이의 감독은 박찬욱입니다.
                  추가 질문: 박찬욱은 어느 나라 출신인가요?
                  중간 답변: 박찬욱은 대한민국 출신입니다.
                  추가 질문: 기생충의 감독은 누구인가요?
                  중간 답변: 기생충의 감독은 봉준호입니다.
                  추가 질문: 봉준호는 어느 나라 출신인가요?
                  중간 답변: 봉준호는 대한민국 출신입니다.
                  최종 답변은: 예
                  """,
    },
]

In [ ]:
example_prompt = PromptTemplate(
    input_variables=["question", "answer"],
    template="Question: {question}\n answer: {answer}"
)
print(example_prompt)

In [ ]:
example_prompt.format(**{"question":"질문 예시", "answer":"답변 예시"})
# example_prompt.format(question="질문 예시", answer="답변 예시") : 같은 형식

In [ ]:
fewshot_prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {question}", # Few Shot Context 생성 후 마지막 부분 형식 정의
    input_variables=["question"], # 프롬프트가 수신할 입력 항목
)
print(fewshot_prompt)

In [ ]:
print(type(fewshot_prompt.format(question="Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?")))

In [ ]:
print(fewshot_prompt.format(question="Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"))

In [ ]:
llm = OpenAI(
    api_key=api_key,
    organization=organization_id
)

chain = llm | StrOutputParser()

In [ ]:
print(chain.invoke("Google이 창립된 연도에 Bill Gates의 나이는 몇 살인가요?"))

# 8. Model의 Parameter 설정

In [ ]:
# 모델 생성 단계에서 설정
params = {
    "temperature": 0.7,
    "max_tokens": 100
}

kwargs = {
    "frequency_penalty": 0.5, # 동일한 단어를 반복해서 사용하지 않도록 페널티를 부여
    "presence_penalty": 0.7, # 이미 등장한 단어를 다시 사용하는 것을 제한
}

llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    **params,
    model_kwargs=kwargs,
    api_key=api_key,
    organization=organization_id
)

question = "태양계에서 가장 큰 행성은 무엇인가요?"
response = llm.invoke(input=question)

print(response.content)

In [ ]:
# 모델 호출 단계에서 설정
params = {
    "temperature": 0.7,
    "max_tokens": 10,
}

response = llm.invoke(input=question, **params)

print(response.content)

In [ ]:
# bind Method를 이용한 파라미터 설정, 특수한 상황에서 일부 파라미터를 다르게 적용 가능
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    max_tokens=100,
    api_key=api_key,
    organization=organization_id
)

prompt = ChatPromptTemplate.from_messages([
    ("system", "이 시스템은 천문학 질문에 답변할 수 있습니다."),
    ("user", "{user_input}"),
])

In [ ]:
messages = prompt.format_messages(user_input="태양계에서 가장 큰 행성은 무엇인가요?")
before_answer = llm.invoke(messages)
print(messages, "\n", before_answer.content)

In [ ]:
chain = prompt | llm.bind(max_tokens=10) | StrOutputParser()

after_answer = chain.invoke({"user_input": "태양계에서 가장 큰 행성은 무엇인가요?"})
print(after_answer)

# 9. 프롬프트 조합과 Partial Prompt Template 사용

In [ ]:
llm = ChatOpenAI(
    model="gpt-3.5-turbo-0125",
    api_key=api_key,
    organization=organization_id
)

### 9-1. Partial Prompt Template
- 프롬프트의 변수 값을 한번에 채워 넣는 것이 아니라, 이 중 일부만 먼저 채워 넣고 나머지는 나중에 채워 넣는 방식

- 애플리케이션 코드내에서 템플릿 생성시 이미 알고 있는 값이 있을때 사용하기 편리한 방식

In [ ]:
output_parser = CommaSeparatedListOutputParser()
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [ ]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}...",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions}
)

In [ ]:
print(prompt)

In [ ]:
print(prompt.format(subject="popular Korean cusine"))

In [ ]:
chain = prompt | llm | output_parser
chain.invoke({"subject":"popular Korean cusine"})

### 9-2. JSON 포맷을 사용하여 출력하는 JsonOutpoutParser
- PydanticOutputParser를 이용하여 출력을 변환하여 원하는 구조 정보로 정의하여 사용

- 원하는 구조정보 정의는 BaseModel과 Field를 이용하여 클래스로 정의하여 전달함

In [ ]:
#자료구조 정의
class CusineRecipe(BaseModel):
    name: str = Field(description="name of a cusine") # name: str -> Type Hinting
    recipe: str = Field(description="recipe to cook the cusine") # recipe: str -> Type Hinting

output_parser = JsonOutputParser(pydantic_object=CusineRecipe)

format_instructions = output_parser.get_format_instructions()
# get_formet_instuctions() : 언어 모델이 출력해야 할 정보의 형식을 정의하는 지침을 제공

print(format_instructions)

In [ ]:
prompt = PromptTemplate(
    template="List five {subject}.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions":format_instructions}
)

chain = prompt | llm | output_parser
print(chain.invoke(input="popular Korean cusine"))

# Appendix. LangChain Agents

In [ ]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    temperature=0,
    api_key=api_key,
    organization=organization_id
)

tools = load_tools(["wikipedia", "llm-math"], llm=llm) # Langchain Tool Load
# https://api.python.langchain.com/en/latest/agents/langchain.agents.load_tools.load_tools.html

agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    description='위키피이아에서 정보를 검색하고 계산이 필요할 때 사용', 
    # Agent 작업의 성격과 용도를 명확히 정의하여 적절한 방법으로 작업을 수행하도록 유도
    verbose=True
)

In [ ]:
print("[result]\n", agent.run("gpt-4o는 언제 출시되었어?"))